In [114]:
import os
import numpy as np
import cv2 as cv
import tensorflow as tf

In [115]:
# 查看是否可以使用GPU

print ("Usage of gpu: {}".format(tf.test.is_gpu_available())) 
print ("Built with CUDA: {}".format(tf.test.is_built_with_cuda()))

Usage of gpu: False
Built with CUDA: False


## 加载训练数据集

In [116]:
def train_value():
  x_train,y_train=[],[]
  k=os.listdir('train_png/')
  for img in k:
    val=cv.imread('train_png/'+img)
    x_train.append(np.array(val))
  k=os.listdir('train_masks_png/')
  for img in k:
    val=cv.imread('train_masks_png/'+img)
    val = cv.cvtColor(val, cv.COLOR_BGR2GRAY)
    res=cv.resize(val,(256,256))
    res=np.expand_dims(res,axis=2)
    y_train.append(np.array(res))
  return x_train,y_train
x_train_img,y_train_mask=train_value()

## 对训练集数据进行标准化

In [117]:
x_train_img=np.asarray(x_train_img)
y_train_mask=np.asarray(y_train_mask)

x_train_img_m=np.mean(x_train_img,axis=0,keepdims=True)
x_train_img_s=np.std(x_train_img,axis=0,keepdims=True)

x_train_img=(x_train_img-x_train_img_m)/x_train_img_s
y_train_mask=y_train_mask*(1./255)
print(x_train_img.shape,y_train_mask.shape)

(12, 256, 256, 3) (12, 256, 256, 1)


## 加载验证数据集

In [118]:
def val_value():
  x_val,y_val, name_list = [],[], []
  k=os.listdir('val_png/')
  for img in k:
    val=cv.imread('val_png/'+img)
    x_val.append(np.array(val))
  k=os.listdir('val_masks_png/')
  for img in k:
    val=cv.imread('val_masks_png/'+img)
    val = cv.cvtColor(val,cv.COLOR_BGR2GRAY)
    res=cv.resize(val,(256,256))
    res=np.expand_dims(res,axis=2)
    y_val.append(np.array(res))
    name_list.append(img)
  return x_val, y_val, name_list
x_val_img,y_val_mask, val_name_list = val_value()

## 对验证集数据进行标准化

In [119]:
x_val_img=np.asarray(x_val_img)
y_val_mask=np.asarray(y_val_mask)

x_val_img_m=np.mean(x_val_img,axis=0,keepdims=True)
x_val_img_s=np.std(x_val_img,axis=0,keepdims=True)

x_val_img=(x_val_img-x_val_img_m)/x_val_img_s
y_val_mask=y_val_mask*(1./255)

print (x_val_img.shape,y_val_mask.shape)

(6, 256, 256, 3) (6, 256, 256, 1)


#### 使用tf.keras构建网络模型

In [120]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

#### 损失函数定义

In [121]:
# dice系数与dice损失函数
# dice 系数
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f**2) + K.sum(y_pred_f**2) + 1)

# dice 损失函数
def dice_loss(y_true,y_pred):
    return 1-dice_coef(y_true,y_pred)

In [122]:
# 二分类交叉熵损失
def binary_crossentropy(y_true,y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)

In [123]:
# 联合损失
def combine_loss(y_true,y_pred):
    return 0.3 * dice_loss(y_true,y_pred) + 0.7 * binary_crossentropy(y_true,y_pred)


#### 网络模型构建

In [124]:
# 构建卷积层：3 X 3卷积 + BN层 +ReLU激活函数
def conv3X3_bn_relu(input, out_channels):

    out = tf.keras.layers.Conv2D(filters=out_channels, kernel_size=(3, 3), strides=(1, 1),
                                     padding="same", kernel_initializer='he_normal')(input)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.activations.relu(out)
    return out

In [125]:
# 构建 AC Block
def ACBlock(input, out_channels):

    out_33 = tf.keras.layers.Conv2D(out_channels, (3, 3), kernel_initializer='he_normal',
                                   padding='same')(input)
    out_33 = tf.keras.layers.BatchNormalization()(out_33)

    out_13 = tf.keras.layers.Conv2D(out_channels, (1, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                   padding='same')(input)
    out_13 = tf.keras.layers.BatchNormalization()(out_13)

    out_31 = tf.keras.layers.Conv2D(out_channels, (3, 1), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                   padding='same')(input)
    out_31 = tf.keras.layers.BatchNormalization()(out_31)

    out = tf.keras.layers.Add()([tf.keras.layers.Add()([out_33, out_13]), out_31])
    out = tf.keras.activations.relu(out)
    return out

In [126]:
# SENet 通道自注意力机制
# 构建 SE Block
def SEBlock(input):

    [N, H, W, C] = input.shape

    avg_input = tf.keras.layers.GlobalAvgPool2D()(input)  # 在特征图的高和宽维度上进行全剧平均值池化
    se_fc_0 = tf.keras.layers.Dense(C // 2, activation=tf.keras.activations.relu)(avg_input)
    se_fc_1 = tf.keras.layers.Dense(C, activation=tf.keras.activations.sigmoid)(se_fc_0)

    out = tf.keras.layers.Multiply()([input, se_fc_1])
    return out

In [127]:
# AC + Att +UNet模型
def AC_Att_UNet(input_img):
    
    map_down_1 = conv3X3_bn_relu(input_img, 64)
    map_down_1 = conv3X3_bn_relu(map_down_1, 64)

    map_down_2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(map_down_1)
    map_down_2 = ACBlock(map_down_2, 128)
    map_down_2 = ACBlock(map_down_2, 128)

    map_down_3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(map_down_2)
    map_down_3 = ACBlock(map_down_3, 256)
    map_down_3 = ACBlock(map_down_3, 256)

    map_down_4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(map_down_3)
    map_down_4 = ACBlock(map_down_4, 512)
    map_down_4 = ACBlock(map_down_4, 512)

    map_down_5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(map_down_4)
    map_down_5 = ACBlock(map_down_5, 1024)
    map_down_5 = ACBlock(map_down_5, 1024)

    map_up_4 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(map_down_5)
    map_down_4 = SEBlock(map_down_4)
    map_up_4 = tf.keras.layers.concatenate([map_up_4, map_down_4])
    map_up_4 = ACBlock(map_up_4, 512)
    map_up_4 = ACBlock(map_up_4, 512)

    map_up_3 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(map_up_4)
    map_down_3 = SEBlock(map_down_3)
    map_up_3 = tf.keras.layers.concatenate([map_up_3, map_down_3])
    map_up_3 = ACBlock(map_up_3, 256)
    map_up_3 = ACBlock(map_up_3, 256)

    map_up_2 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(map_up_3)
    map_down_2 = SEBlock(map_down_2)
    map_up_2 = tf.keras.layers.concatenate([map_up_2, map_down_2])
    map_up_2 = ACBlock(map_up_2, 256)
    map_up_2 = ACBlock(map_up_2, 256)

    map_up_1 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(map_up_2)
    map_down_1 = SEBlock(map_down_1)
    map_up_1 = tf.keras.layers.concatenate([map_up_1, map_down_1])
    map_up_1 = ACBlock(map_up_1, 256)
    map_up_1 = ACBlock(map_up_1, 256)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(map_up_1)
    model = Model(inputs=input_img, outputs=outputs)
    return model

# 准备训练

In [128]:
# 初始化模型
model=AC_Att_UNet(input_img=Input(shape=(256,256,3)))
# print("AC_Att_UNet...")
# model.summary()  # 打印模型

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 256, 256, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_114 (BatchN (None, 256, 256, 64) 256         conv2d_116[0][0]                 
__________________________________________________________________________________________________
tf_op_layer_Relu_41 (TensorFlow [(None, 256, 256, 64 0           batch_normalization_114[0][0]    
____________________________________________________________________________________________

In [129]:
# 使用Adam优化器，联合损失函数，评估指标使用准确率和dice系数
LEARNING_RATE = 0.01  # 学习率
adam = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam',loss=combine_loss,metrics= ['accuracy', dice_coef])

In [130]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
model_path="{}_best.hdf5".format('my_model')
# 保存模型
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1,save_best_only=True, mode='min', save_weights_only = False)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3,verbose=1, mode='min', epsilon=0.0001, cooldown=1, min_lr=1e-7)
early = EarlyStopping(monitor="val_loss",mode="min",patience=7) 
callbacks_list = [checkpoint, early, reduceLROnPlat]

## 开始训练

In [131]:
# 模型训练
BATCHSIZE = 1  # batch size
EPOCHS = 2  # epochs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen_args = dict(rotation_range=90,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2)

image_datagen = ImageDataGenerator(**data_gen_args)  # 产生数据

model.fit(image_datagen.flow(x_train_img,y_train_mask,batch_size=BATCHSIZE),
          epochs=EPOCHS,
          callbacks=callbacks_list,
          validation_data=(x_val_img,y_val_mask))

# 每训练完成一个epoch，在验证集上验证一次，评价指标使用dice系数和accuracy
# 由于keras的训练阶段评价指标中不包含精度、召回率和 F1分数，所以在后边有专门的定义

  ...
    to  
  ['...']
Train for 12 steps, validate on 6 samples
Epoch 1/2
11/12 [==========================>...] - ETA: 11s - loss: 0.4448 - accuracy: 0.8477 - dice_coef: 0.9117 

  ...
    to  
  ['...']


## 训练完成，开始在验证集上验证

In [132]:
# 加载模型，modell为加载完参数的模型，准备验证
from tensorflow.keras.models import load_model
modelL=load_model('my_model_best.hdf5',
                  custom_objects={'combine_loss':combine_loss,
                                  'dice_coef':dice_coef})

Done.


In [133]:
# 模型评估：percision，recal，F1-score
SAVE_PRED = True  # 是否保存分割结果图
SAVE_PATH = "results"  # 保存路径
from sklearn.metrics import f1_score, precision_score, recall_score
from skimage import io
import os.path as osp

val_dataset_size = x_val_img.shape[0]
recall_list = np.array([0.0] * val_dataset_size)
precision_list = np.array([0.0] * val_dataset_size)
F1_list = np.array([0.0] * val_dataset_size)

print("开始推理...")
for i in range(val_dataset_size):
    print("推理{}的分割掩膜...".format(val_name_list[i]))
    val_img = x_val_img[i]
    val_mask = y_val_mask[i]
    y_true = np.squeeze(val_mask)
    final_val_img = np.expand_dims(val_img,axis=0)
    pred = modelL.predict(final_val_img)
    y_pred=np.squeeze(pred)
    y_pred[y_pred>0.5]=1.0
    y_pred[y_pred<=0.5]=0.0   # 阈值为 0.5，大于 0.5 的为类别 1，否则为类别 0
    if SAVE_PRED:
        save_pred = y_pred * 255.0
        save_pred = save_pred.astype(np.uint8)
        io.imsave(osp.join(SAVE_PATH, val_name_list[i]), save_pred)
    y_true = np.reshape(y_true, newshape=(-1))
    y_pred = np.reshape(y_pred, newshape=(-1))
    recall_list[i] = recall_score(y_true, y_pred)
    precision_list[i] = precision_score(y_true, y_pred)
    F1_list[i] = f1_score(y_true, y_pred)

print("*************************************************************")
print('对验证集的测试结果：\nrecall:%.3f\tprecision:%.3f\tF1-score:%.3f'%
    (np.mean(recall_list), np.mean(precision_list), np.mean(F1_list)))
print("*************************************************************")

开始推理...
推理7707.png的分割掩膜...
推理7785.png的分割掩膜...
推理7786.png的分割掩膜...
推理7850.png的分割掩膜...
推理7928.png的分割掩膜...
推理8130.png的分割掩膜...
*************************************************************
对验证集的测试结果：
recall:1.000	precision:0.925	F1-score:0.960
*************************************************************


E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\7707.png is a low contrast image
E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\7785.png is a low contrast image
E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\7786.png is a low contrast image
E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\7850.png is a low contrast image
E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\7928.png is a low contrast image
E:\anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: results\8130.png is a low contrast image
